In [1]:
import pandas as pd
import numpy as np
import requests
import os
import time
import finviz as fz
import re
import requests
import unicodedata
from bs4 import BeautifulSoup

In [2]:
years = [2000 + x for x in range(0,21)]
quarters = ['QTR1', 'QTR2', 'QTR3', 'QTR4']

base_path = 'C:\\Users\\hotal\\Documents\\DSC540\\Finals'

current_dirs = os.listdir(path=base_path)

for yr in years:
    if str(yr) not in current_dirs:
        os.mkdir('/'.join([base_path, str(yr)]))
    current_files = os.listdir('/'.join([base_path, str(yr)]))
    
    for qtr in quarters:
        local_filename =  f'xbrl-index-{yr}-{qtr}.txt'
        local_file_path = '/'.join([base_path, str(yr), local_filename])
        if local_filename in current_files:
            #print(f'Skipping index file for {yr}, {qtr} because it is already saved.')
            continue
        url = f'https://www.sec.gov/Archives/edgar/full-index/{yr}/{qtr}/xbrl.idx'
        r = requests.get(url, stream=True)
        with open(local_file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=10240):
                f.write(chunk)
        time.sleep(10)

In [3]:
def ticker_downloader():
    tickers = pd.read_table("https://www.sec.gov/include/ticker.txt", names = ['Ticker', "CIK"])
    tickers.to_csv("ticker.csv")
    
ticker_downloader()

In [4]:
def ticker_to_cik(ticker):
    """Returns the CIK value for a given ticker"""
    tickers = pd.read_csv("ticker.csv")
    ticker = ticker.replace(".","-")
    return int(tickers[tickers['Ticker'] == ticker.lower()]["CIK"])

In [77]:
super_frame = pd.DataFrame(columns = ["CIK", "Company Name", "Form Type","Date Filed","Filename"])

for i in range(2000,2021):
    for j in range(1,5):
        df = pd.read_table("{}/xbrl-index-{}-QTR{}.txt".format(i,i,j),sep="|", skiprows=10, 
                           names = ["CIK", "Company Name", "Form Type","Date Filed","Filename"])
        super_frame = super_frame.append(df, ignore_index=True)

In [82]:
desired_ticker = "AAPL"

super_frame[(super_frame['Form Type'] == "10-K") & (super_frame['CIK'] == ticker_to_cik(desired_ticker))]

,CIK,Company Name,Form Type,Date Filed,Filename
1530,320193,APPLE INC,10-K,2009-10-27,edgar/data/320193/0001193125-09-214859.txt
5254,320193,APPLE INC,10-K,2010-10-27,edgar/data/320193/0001193125-10-238044.txt
26066,320193,APPLE INC,10-K,2011-10-26,edgar/data/320193/0001193125-11-282113.txt
65078,320193,APPLE INC,10-K,2012-10-31,edgar/data/320193/0001193125-12-444068.txt
104095,320193,APPLE INC,10-K,2013-10-30,edgar/data/320193/0001193125-13-416534.txt
141960,320193,APPLE INC,10-K,2014-10-27,edgar/data/320193/0001193125-14-383437.txt
178067,320193,APPLE INC,10-K,2015-10-28,edgar/data/320193/0001193125-15-356351.txt
243095,320193,APPLE INC,10-K,2016-10-26,edgar/data/320193/0001628280-16-020309.txt
284704,320193,APPLE INC,10-K,2017-11-03,edgar/data/320193/0000320193-17-000070.txt
316995,320193,APPLE INC,10-K,2018-11-05,edgar/data/320193/0000320193-18-000145.txt


In [81]:
pd.DataFrame(fz.get_insider(desired_ticker)).head(5)

,Insider Trading,Relationship,Date,Transaction,Cost,#Shares,Value ($),#Shares Total,SEC Form 4
0,Karsner Alexander,Director,Jun 22,Sale,60.50,"3,467","209,754","7,508",Jun 22 07:59 PM
1,Karsner Alexander,Director,Jun 18,Sale,60.43,"20,027","1,210,232","10,975",Jun 22 07:59 PM
2,Forrest Stephen R,Director,Feb 14,Sale,67.40,"2,000","134,800","70,955",Feb 19 08:00 PM
3,Raja Prabu G.,"SVP, Semi. Products Group",Dec 16,Sale,60.73,"50,000","3,036,500","504,329",Dec 18 08:00 PM


In [55]:
base_url = "https://www.sec.gov"

In [56]:
normal_url = "https://www.sec.gov/Archives/edgar/data/1326801/0001326801-17-000007.txt"

In [57]:
normal_url = normal_url.replace("-","").replace(".txt",'/index.json')
normal_url

'https://www.sec.gov/Archives/edgar/data/1326801/000132680117000007/index.json'

In [58]:
content = requests.get(normal_url).json()

In [59]:
pd.DataFrame(content)

,directory
item,"[{'last-modified': '2017-02-02 21:13:52', 'nam..."
name,/Archives/edgar/data/1326801/000132680117000007
parent-dir,/Archives/edgar/data/1326801


In [60]:
for file in content['directory']['item']:
    if file['name'] == 'FilingSummary.xml':
        xml_summary = base_url + content['directory']['name'] + "/" + file['name']
        print(xml_summary)
        print("-"*100)
        print("File Name: " + file['name'])
        print("File Path: " + xml_summary)

https://www.sec.gov/Archives/edgar/data/1326801/000132680117000007/FilingSummary.xml
----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/1326801/000132680117000007/FilingSummary.xml


In [61]:
base_url = xml_summary.replace('FilingSummary.xml',"")
content = requests.get(xml_summary).content
soup = BeautifulSoup(content, 'lxml')

In [62]:
reports = soup.find('myreports')

In [63]:
master_reports = []

for report in reports.find_all('report')[:-1]:
    report_dict = {}
    report_dict['name_short'] = report.shortname.text
    report_dict['name_long'] = report.longname.text
    report_dict['position'] = report.position.text
    report_dict['category'] = report.menucategory.text
    report_dict['url'] = base_url + report.htmlfilename.text
    
    master_reports.append(report_dict)
    

In [64]:
testdf = pd.DataFrame(master_reports)

In [65]:
testdf

,name_short,name_long,position,category,url
0,Document and Entity Information,0001000 - Document - Document and Entity Infor...,1,Cover,https://www.sec.gov/Archives/edgar/data/132680...
1,CONSOLIDATED BALANCE SHEETS,1001000 - Statement - CONSOLIDATED BALANCE SHEETS,2,Statements,https://www.sec.gov/Archives/edgar/data/132680...
2,CONSOLIDATED BALANCE SHEETS (Parenthetical),1001501 - Statement - CONSOLIDATED BALANCE SHE...,3,Statements,https://www.sec.gov/Archives/edgar/data/132680...
3,CONSOLIDATED STATEMENTS OF INCOME,1002000 - Statement - CONSOLIDATED STATEMENTS ...,4,Statements,https://www.sec.gov/Archives/edgar/data/132680...
4,CONSOLIDATED STATEMENTS OF COMPREHENSIVE INCOME,1003000 - Statement - CONSOLIDATED STATEMENTS ...,5,Statements,https://www.sec.gov/Archives/edgar/data/132680...
...,...,...,...,...,...
65,Income Taxes - Deferred Tax Assets and Liabil...,2414405 - Disclosure - Income Taxes - Deferre...,66,Details,https://www.sec.gov/Archives/edgar/data/132680...
66,Income Taxes - Unrecognized Tax Benefits (Det...,2414406 - Disclosure - Income Taxes - Unrecog...,67,Details,https://www.sec.gov/Archives/edgar/data/132680...
67,Income Taxes - Narrative (Detail),2414407 - Disclosure - Income Taxes - Narrati...,68,Details,https://www.sec.gov/Archives/edgar/data/132680...
68,Geographical Information - Revenue (Details),2415402 - Disclosure - Geographical Informatio...,69,Details,https://www.sec.gov/Archives/edgar/data/132680...


In [66]:
statements_url = []

for report_dict in master_reports:
    """I should use REGEX for this part since most will be differently named"""
    
    item1 = "Consolidated Balance Sheets".upper()
    item2 = "CONSOLIDATED STATEMENTS OF INCOME".upper()
    item3 = "Consolidated Statements of Cash Flows".upper()
    item4 = "Consolidated Statements of Stockholders' Equity".upper()
    
    report_list = [item1,item2,item3,item4]
    
    if report_dict['name_short'].upper() in report_list:
        print(report_dict['name_short'])
        print(report_dict['url'])
        statements_url.append(report_dict['url'])

CONSOLIDATED BALANCE SHEETS
https://www.sec.gov/Archives/edgar/data/1326801/000132680117000007/R2.htm
CONSOLIDATED STATEMENTS OF INCOME
https://www.sec.gov/Archives/edgar/data/1326801/000132680117000007/R4.htm
CONSOLIDATED STATEMENTS OF STOCKHOLDERS' EQUITY
https://www.sec.gov/Archives/edgar/data/1326801/000132680117000007/R6.htm
CONSOLIDATED STATEMENTS OF CASH FLOWS
https://www.sec.gov/Archives/edgar/data/1326801/000132680117000007/R7.htm


In [67]:
statements_data = []

for statement in statements_url:
    statement_data = {}
    statement_data['header'] = []
    statement_data['section'] = []
    statement_data['data'] = []
    content = requests.get(statement).content
    report_soup = BeautifulSoup(content,"html")
    for index, row in enumerate(report_soup.table.find_all('tr')):
        cols = row.find_all('td')
        if (len(row.find_all('th'))) == 0 and (len(row.find_all('strong')) == 0):
            reg_row = [ele.text.strip() for ele in cols]
            statement_data['data'].append(reg_row)
        elif (len(row.find_all('th')) == 0 and len(row.find_all('strong')) != 0):
            sec_row = cols[0].text.strip()
            statement_data['section'].append(sec_row)
        elif (len(row.find_all('th'))!= 0):
            hed_row = [ele.text.strip() for ele in row.find_all('th')]
            statement_data['header'].append(hed_row)
    statements_data.append(statement_data)

In [21]:
income_header = statements_data[1]['header'][1]
income_data = statements_data[1]['data']
''
income_df = pd.DataFrame(income_data, columns=[["Type"] + income_header])
income_df

,Type,"Dec. 31, 2019","Dec. 31, 2018","Dec. 31, 2017"
0,Revenue,"$ 70,697","$ 55,838","$ 40,653"
1,Cost of revenue,"12,770","9,355","5,454"
2,Research and development,"13,600","10,273","7,754"
3,Marketing and sales,"9,876","7,846","4,725"
4,General and administrative,"10,465","3,451","2,517"
5,Total costs and expenses,"46,711","30,925","20,450"
6,Income from operations,"23,986","24,913","20,203"
7,"Interest and other income, net",826,448,391
8,Income before provision for income taxes,"24,812","25,361","20,594"
9,Provision for income taxes,"6,327","3,249","4,660"


In [68]:
income_header = statements_data[1]['header'][1]
income_data = statements_data[1]['data']
''
income_df = pd.DataFrame(income_data, columns=[["Type"] + income_header])
income_df

,Type,"Dec. 31, 2016","Dec. 31, 2015","Dec. 31, 2014"
0,Revenue,"$ 27,638","$ 17,928","$ 12,466"
1,Cost of revenue,"3,789","2,867","2,153"
2,Research and development,"5,919","4,816","2,666"
3,Marketing and sales,"3,772","2,725","1,680"
4,General and administrative,"1,731","1,295",973
5,Total costs and expenses,"15,211","11,703","7,472"
6,Income from operations,"12,427","6,225","4,994"
7,"Interest and other income/(expense), net",91,(31),(84)
8,Income before provision for income taxes,"12,518","6,194","4,910"
9,Provision for income taxes,"2,301","2,506","1,970"
